In [31]:

import cv2
import numpy as np
from matplotlib import pyplot as plt

imgo = cv2.imread("NewImage-1.png", cv2.IMREAD_GRAYSCALE)
img = cv2.threshold(imgo,128,255,cv2.THRESH_BINARY)[1]
print(img)
print(img.shape)
cv2.namedWindow('img', cv2.WINDOW_NORMAL)
cv2.resizeWindow('img', 600, 400)
cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

[[  0   0   0   0   0 255 255 255 255 255]
 [  0   0   0   0   0 255 255 255 255 255]
 [  0   0   0   0   0 255 255 255 255 255]
 [  0   0   0   0   0 255 255 255 255 255]
 [  0   0   0   0   0 255 255 255 255 255]
 [255 255 255 255 255   0   0   0   0   0]
 [255 255 255 255 255 255 255 255 255 255]
 [255 255 255 255 255 255 255 255 255 255]
 [255 255 255 255 255 255 255 255 255 255]
 [255 255 255 255 255 255 255 255 255 255]]
(10, 10)


In [32]:
def split_img(img,block_h,block_w):
    blocks = []
    h , w = img.shape[:2]
    if h%block_h!=0:
        print("invalid block height")
        return
    if w%block_w!=0:
        print("invalid block width")
        return
    num_rows = w//block_w
    num_cols = h//block_h
    for y in range(num_cols):
        for x in range(num_rows):
                left = x * block_w
                upper = y * block_h
                right = left + block_w
                lower = upper + block_h
                block = img[upper:lower, left:right]
                blocks.append(block)
    return blocks

In [33]:
def CAC_encode(img,block_h,block_w):    
    blocks = split_img(img, block_w, block_h)
    print(blocks)
    print("Image size:",img.size)
    codes = ['0','11','10']
    code = []
    for block in blocks:
        if np.all(block == 0):
            code.append(codes[0])
        elif np.all(block == 255):
            code.append(codes[1])
        else:
            code.append(codes[2])
            block = block.tolist()
            for blk in block:
                for b in blk:
                    if b == 255:
                        code.append(1)   
                    else:
                        code.append(b)
    code ="".join(map(str,code))
    print("Code length: ", len(code))
    
    CR = img.size/len(code)
    RD = 1 - 1/CR
    print("Compression Ratio: " , CR , " \nRelative Data Redundancy: ", RD) 
    return  block_h ,block_w , img.shape[:2], CR , RD , code

In [34]:
CAC_encode(img,5,5)

[array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]], dtype=uint8), array([[255, 255, 255, 255, 255],
       [255, 255, 255, 255, 255],
       [255, 255, 255, 255, 255],
       [255, 255, 255, 255, 255],
       [255, 255, 255, 255, 255]], dtype=uint8), array([[255, 255, 255, 255, 255],
       [255, 255, 255, 255, 255],
       [255, 255, 255, 255, 255],
       [255, 255, 255, 255, 255],
       [255, 255, 255, 255, 255]], dtype=uint8), array([[  0,   0,   0,   0,   0],
       [255, 255, 255, 255, 255],
       [255, 255, 255, 255, 255],
       [255, 255, 255, 255, 255],
       [255, 255, 255, 255, 255]], dtype=uint8)]
Image size: 100
Code length:  32
Compression Ratio:  3.125  
Relative Data Redundancy:  0.6799999999999999


(5, 5, (10, 10), 3.125, 0.6799999999999999, '01111100000011111111111111111111')

In [35]:
def CAC_reblock(code,block_h,block_w,height,width):
    codes = ['0','11','10']
    blocks = []
    i = 0
    while i in range(len(code)) :
        if code[i] == codes[0]:
            blocks.append(np.zeros((block_h, block_w), dtype=np.uint8))
            i=i+1
        elif code[i:i+2] == codes[1]:
            blocks.append(np.ones((block_h, block_w), dtype=np.uint8) * 255)
            i = i+2
        elif code[i:i+2] == codes[2]: 
            i=i+2
            arr = np.zeros((block_h, block_w), dtype=np.uint8)
            for r in range(0,block_h):
                for c in range(0,block_w):
                    arr[r][c] = int(code[i])
                    i = i+1
            blocks.append(arr * 255)
    print(blocks)

In [36]:
def CAC_decode(code,block_h,block_w,height,width):
    img = np.ones((height, width), dtype=np.uint8)
    codes = ['0','11','10']
    blocks = []
    i = 0
    x = 0
    y = 0
    while i in range(len(code)):
        if x >= width:
            y = y + block_h 
            x = 0
        
        if code[i] == codes[0]:
            for r in range(0,block_h):
                for c in range(0,block_w):
                    img[r+y][c+x] = 0
            x = x + block_w
            i = i+1
            
        elif code[i:i+2] == codes[1]:
            i = i+2
            for r in range(0,block_h):
                for c in range(0,block_w):
                    img[r+y][c+x] = 255
            x= x + block_w
            
        elif code[i:i+2] == codes[2]: 
            i = i+2
            for r in range(0,block_h):
                for c in range(0,block_w):
                    img[r+y][c+x] = int(code[i]) *255
                    i = i+1
            x = x + block_w
    return img

In [37]:
code = str('01111100000011111111111111111111')
block_h = 5
block_w = 5
height = 10
width = 10

img = CAC_decode(code,block_h,block_w,height,width)
print(img)

img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
cv2.namedWindow('img', cv2.WINDOW_NORMAL)
cv2.resizeWindow('img', 600, 400)
cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

[[  0   0   0   0   0 255 255 255 255 255]
 [  0   0   0   0   0 255 255 255 255 255]
 [  0   0   0   0   0 255 255 255 255 255]
 [  0   0   0   0   0 255 255 255 255 255]
 [  0   0   0   0   0 255 255 255 255 255]
 [255 255 255 255 255   0   0   0   0   0]
 [255 255 255 255 255 255 255 255 255 255]
 [255 255 255 255 255 255 255 255 255 255]
 [255 255 255 255 255 255 255 255 255 255]
 [255 255 255 255 255 255 255 255 255 255]]
